In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime

#* This block of code fetches the yfinance history data (until the present) for the AAPL Stock Share
#* if not in existance, else pass

print(r'Creating AAPL instance...', end='')
aapl = yf.Ticker('TSLA')
hist = aapl.history(period='max')
print('\rAAPL instance Created      ')

AAPL instance Created      


In [2]:
df = pd.DataFrame(hist)
df = df.reset_index()
df.columns = [_.lower() for _ in df.columns]
df['openclose_avr'] = (df['open'] + df['close']) / 2
df['datetime'] = df['date'].dt.date
df['body'] = abs(df['open'] - df['close'])
df['upper_shadow'] = df['high'] - df[['open', 'close']].max(axis=1)
df['lower_shadow'] = df[['open', 'close']].min(axis=1) - df['low']

# Conditions for detecting various types of hammers
condition_bullish_hammer = (df['body'] < 0.1 * df['open']) & (df['lower_shadow'] >= 2 * df['body']) & (df['upper_shadow'] <= 0.1 * df['open'])
condition_inverted_hammer = (df['body'] < 0.1 * df['open']) & (df['upper_shadow'] >= 2 * df['body']) & (df['lower_shadow'] <= 0.1 * df['open'])

# Create a new column 'hammer' based on the conditions
df['hammer'] = condition_bullish_hammer
df['inv_hammer'] = condition_inverted_hammer
df.head()

,date,open,high,low,close,volume,dividends,stock splits,openclose_avr,datetime,body,upper_shadow,lower_shadow,hammer,inv_hammer
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0,1.429667,2010-06-29,0.326000,0.074000,0.097334,False,False
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0,1.654000,2010-06-30,0.130666,0.308667,0.035334,False,True
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0,1.565333,2010-07-01,0.202667,0.061333,0.112667,False,False
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0,1.406666,2010-07-02,0.253333,0.006667,0.032667,False,False
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0,1.203667,2010-07-06,0.259333,0.000000,0.018667,False,False


In [3]:
sample = df.iloc[:-50]
sample.head()

,date,open,high,low,close,volume,dividends,stock splits,openclose_avr,datetime,body,upper_shadow,lower_shadow,hammer,inv_hammer
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0,1.429667,2010-06-29,0.326000,0.074000,0.097334,False,False
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0,1.654000,2010-06-30,0.130666,0.308667,0.035334,False,True
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0,1.565333,2010-07-01,0.202667,0.061333,0.112667,False,False
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0,1.406666,2010-07-02,0.253333,0.006667,0.032667,False,False
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0,1.203667,2010-07-06,0.259333,0.000000,0.018667,False,False


In [4]:
#! If you want to plot with matplotlib
# %matplotlib inline
# import matplotlib.pyplot as plt
# up_mask = sample['close'] > sample['open']
# up = sample[up_mask]
# down = sample[~up_mask]

# width = 0.4
# width2 = 0.05

# with plt.style.context('dark_background'):
#     plt.figure(figsize=(15,7.5))
#     plt.plot(sample['openclose_avr'], color='white', alpha=0.75)
    
# #* Plot up prices
#     plt.bar(up.index, up['close']-up['open'], width, bottom=up['open'], color='green')
#     plt.bar(up.index, up['high']-up['close'], width2, bottom=up['close'], color='green')
#     plt.bar(up.index, up['low']-up['open'], width2, bottom=up['open'], color='green')

# #* Plot down prices
#     plt.bar(down.index, down['close']-down['open'], width, bottom=down['open'], color='red')
#     plt.bar(down.index, down['high']-down['close'], width2, bottom=down['close'], color='red')
#     plt.bar(down.index, down['low']-down['open'], width2, bottom=down['open'], color='red')
    

In [6]:
from bokeh.io import curdoc, output_notebook
from bokeh.plotting import figure, show

#* To plot the 1000 most recent values of df
df_plot = sample
df_plot = df.iloc[-100:]

#* Settings
output_notebook()
curdoc().theme = "dark_minimal"

up_mask = df_plot['close'] > df_plot['open']
up = df_plot[up_mask]
down = df_plot[~up_mask]
hammer = df_plot[df_plot['hammer']]
inv_hammer = df_plot[df_plot['inv_hammer']]
w = 12*60*60*1000

fig = figure(width=1250, height=650, x_axis_type="datetime")

#* Plot up prices
color = 'lightgreen'
fig.vbar(x = up['datetime'], bottom = up['low'], top = up['high'], line_color = color)
fig.vbar(x = up['datetime'], bottom = up['open'], top = up['close'], color = color, width = w, legend_label = 'Up')

#* Plot down prices
color = 'crimson'
fig.vbar(x = down['datetime'], bottom = down['low'], top = down['high'], line_color = color)
fig.vbar(x = down['datetime'], bottom = down['open'], top = down['close'], color = color, width = w, legend_label = 'Down')

fig.line(df_plot['datetime'], df_plot['openclose_avr'], color='lightgray', legend_label = 'Up/Down Average')
#fig.circle(df_plot['datetime'], df_plot['openclose_avr'], color = 'gray', alpha=0.5)

#* Plot possible direction change
color = 'gold'

    #! Normal hammer:
fig.vbar(x = hammer['datetime'], bottom = hammer['low'], top = hammer['high'], line_color = 'green')
fig.vbar(x = hammer['datetime'], bottom = hammer['open'], top = hammer['close'], color = color, width = w, legend_label = 'Hammer')

    #! Inverted hammer:
fig.vbar(x = inv_hammer['datetime'], bottom = inv_hammer['low'], top = inv_hammer['high'], line_color = 'red')
fig.vbar(x = inv_hammer['datetime'], bottom = inv_hammer['open'], top = inv_hammer['close'], color = color, width = w)

# * Show plot
show(fig)

Loading BokehJS ...